### Selected samples SiNET validation

**Author:** Jakub Walczak, PhD

This notebook contains validation of the proposed SiNET method against 
kriging and IDW for a set of validation samples.

The notebook was used to upgrade the design of SiNET.

In [1]:
import csv
import shutil
from functools import partial
from pathlib import Path
from typing import Any, Callable

import xarray as xr
from bayes_opt import BayesianOptimization
from rich.console import Console

import climatrix as cm

%load_ext rich

In [2]:
console = Console()

use_elevation = {0: False, 1: True}

INF_LOSS = -1e4

NAN_POLICY = "resample"
console.print("[bold green]Using NaN policy: [/bold green]", NAN_POLICY)

SEED = 1
console.print("[bold green]Using seed: [/bold green]", SEED)

DSET_PATH = Path(__session__).parent.parent.joinpath("data")
console.print("[bold green]Using dataset path: [/bold green]", DSET_PATH)

EUROPE_BOUNDS = {"north": 71, "south": 36, "west": -24, "east": 35}
EUROPE_DOMAIN = cm.Domain.from_lat_lon(
    lat=slice(EUROPE_BOUNDS["south"], EUROPE_BOUNDS["north"], 0.1),
    lon=slice(EUROPE_BOUNDS["west"], EUROPE_BOUNDS["east"], 0.1),
    kind="dense",
)
cm.seed_all(SEED)

Using NaN policy:  resample

Using seed:  1

Using dataset path:  /storage/tul/projects/climatrix/experiments/jwalczak/01_Apr_02_compare_recon_method/data

In [3]:
def get_all_dataset_idx() -> list[str]:
    return sorted(
        list({path.stem.split("_")[-1] for path in DSET_PATH.glob("*.nc")})
    )

In [4]:
def run_single_method(
    d: str, i: int, method: str, reconstruct_dense: bool = True, **params
):
    cm.seed_all(SEED)
    train_dset = xr.open_dataset(
        DSET_PATH / f"ecad_obs_europe_train_{d}.nc"
    ).cm
    val_dset = xr.open_dataset(DSET_PATH / f"ecad_obs_europe_val_{d}.nc").cm
    reconstructed_dset = train_dset.reconstruct(
        val_dset.domain,
        method=method,
        checkpoint="./checkpoint",
        overwrite_checkpoint=True,
        validation=val_dset,
        **params,
    )
    if reconstruct_dense:
        reconstructed_dense = train_dset.reconstruct(
            EUROPE_DOMAIN, method=method, checkpoint="./checkpoint", **params
        )
    return val_dset, reconstructed_dset, reconstructed_dense

In [5]:
dset_idx = get_all_dataset_idx()
console.print(
    f"[bold green]There is [bold yellow]{len(dset_idx)}[/bold yellow] samples available [/bold green]"
)

There is 100 samples available 

In [6]:
IDX = 0

In [7]:
sinet_val_dset, sinet_reconstructed_dset, sinet_reconstructed_dense = (
    run_single_method(
        dset_idx[IDX],
        IDX,
        "sinet",
        lr=1e-3,
        num_epochs=400,
        batch_size=20,
        num_workers=0,
        device="cuda",
        gradient_clipping_value=1e2,
        mse_loss_weight=1.0,
        eikonal_loss_weight=1e-4,
        laplace_loss_weight=1e-4,
        patience=100,
    )
)

29-07-2025 10:38:54 INFO | climatrix.reconstruct.sinet.sinet | Using checkpoint path: /storage/tul/projects/climatrix/experiments/jwalczak/01_Apr_02_compare_recon_method/notebooks/checkpoint
29-07-2025 10:38:54 INFO | climatrix.reconstruct.sinet.sinet | Initializing SiNET model...
29-07-2025 10:38:54 INFO | climatrix.reconstruct.sinet.sinet | Configuring Adam optimizer with learning rate: 0.001000
29-07-2025 10:38:55 INFO | climatrix.reconstruct.sinet.sinet | Training SiNET model...
29-07-2025 10:39:14 INFO | climatrix.reconstruct.sinet.sinet | Reconstructing target domain...
29-07-2025 10:39:14 INFO | climatrix.reconstruct.sinet.sinet | Creating mini-batches for surface reconstruction...
29-07-2025 10:39:14 INFO | climatrix.reconstruct.sinet.sinet | Processing mini-batch 1/1...
29-07-2025 10:39:14 INFO | climatrix.reconstruct.sinet.sinet | Surface finding complete. Concatenating results.
29-07-2025 10:39:14 INFO | climatrix.reconstruct.sinet.sinet | Using checkpoint path: /storage/tul

In [8]:
cm.Comparison(sinet_val_dset, sinet_reconstructed_dset).compute_report()


{
    'RMSE': 2.8415279388427734,
    'MAE': 1.4909709692001343,
    'Max Abs Error': 11.992989540100098,
    'R^2': 0.5366346836090088
}

### After optimising hyperpararmeters

In [9]:
BOUNDS = {
    "lr": (1e-5, 1e-2),
    "num_epochs": (50, 500),
    "gradient_clipping_value": (1e-4, 1e4),
    "batch_size": (32, 1024),
    "mse_loss_weight": (1e-5, 1e5),
    "eikonal_loss_weight": (0, 1.0),
    "laplace_loss_weight": (0, 1.0),
    "patience": (10, 200),
    "use_elevation": ("0", "1"),  # 0 for False, 1 for True
    "scale": (1e-4, 1e2),
    "layers": (1, 10),
    "hidden_dim": (1, 32),
    "sorting_group_size": (1, 8)
}
console.print("[bold green]Hyperparameter bounds: [/bold green]", BOUNDS)

OPTIM_INIT_POINTS: int = 50
console.print(
    "[bold green]Using nbr initial points for optimization: [/bold green]",
    OPTIM_INIT_POINTS,
)

OPTIM_N_ITERS: int = 100
console.print(
    "[bold green]Using iterations for optimization[/bold green]", OPTIM_N_ITERS
)

Hyperparameter bounds: 
{
    'lr': (1e-05, 0.01),
    'num_epochs': (50, 500),
    'gradient_clipping_value': (0.0001, 10000.0),
    'batch_size': (32, 1024),
    'mse_loss_weight': (1e-05, 100000.0),
    'eikonal_loss_weight': (0, 1.0),
    'laplace_loss_weight': (0, 1.0),
    'patience': (10, 200),
    'use_elevation': ('0', '1'),
    'scale': (0.0001, 100.0),
    'layers': (1, 10),
    'hidden_dim': (1, 32),
    'sorting_group_size': (1, 8)
}

Using nbr initial points for optimization:  50

Using iterations for optimization 100

In [10]:
def compute_criterion(
    train_dset: cm.BaseClimatrixDataset,
    val_dset: cm.BaseClimatrixDataset,
    **hparams,
) -> float:
    cm.seed_all(SEED)
    lr = float(hparams["lr"])
    num_epochs = int(hparams["num_epochs"])
    gradient_clipping_value = float(hparams["gradient_clipping_value"])
    batch_size = int(hparams["batch_size"])
    mse_loss_weight = float(hparams["mse_loss_weight"])
    eikonal_loss_weight = float(hparams["eikonal_loss_weight"])
    laplace_loss_weight = float(hparams["laplace_loss_weight"])
    patience = int(hparams["patience"])
    use_elevation = bool(int(hparams["use_elevation"]))
    scale=float(hparams["scale"])
    layers=int(hparams["layers"])
    sorting_group_size=int(hparams["sorting_group_size"])
    hidden_dim=sorting_group_size*int(hparams["hidden_dim"])
    recon_dset = train_dset.reconstruct(
        val_dset.domain,
        method="sinet",
        layers=layers,
        scale=scale,
        hidden_dim=hidden_dim,
        sorting_group_size=sorting_group_size,
        lr=lr,
        num_epochs=num_epochs,
        batch_size=batch_size,
        num_workers=0,
        device="cuda",
        gradient_clipping_value=gradient_clipping_value,
        mse_loss_weight=mse_loss_weight,
        eikonal_loss_weight=eikonal_loss_weight,
        laplace_loss_weight=laplace_loss_weight,
        patience=patience,
        use_elevation=use_elevation,
    )
    metrics = cm.Comparison(recon_dset, val_dset).compute_report()
    # NOTE: minus to force maximizing
    return -metrics["MAE"]


def find_hyperparameters(
    train_dset: cm.BaseClimatrixDataset,
    val_dset: cm.BaseClimatrixDataset,
    func: Callable[
        [cm.BaseClimatrixDataset, cm.BaseClimatrixDataset, dict], float
    ],
    bounds: dict[str, tuple],
    n_init_points: int = 30,
    n_iter: int = 200,
    seed: int = 0,
    verbose: int = 2,
) -> tuple[float, dict[str, float]]:
    """
    Find hyperparameters using Bayesian Optimization.

    Parameters
    ----------
    train_dset : cm.BaseClimatrixDataset
        Training dataset.
    val_dset : cm.BaseClimatrixDataset
        Validation dataset.
    func : Callable
        Function to optimize.
        It should take two datasets and a dictionary of hyperparameters,
        and return a float score.
    bounds : dict[str, tuple]
        Dictionary of hyperparameter bounds.
        Keys are hyperparameter names, values are tuples (min, max).
    n_init_points : int, optional
        Number of initial random points to sample, by default 30.
    n_iter : int, optional
        Number of iterations for optimization, by default 200.
    seed : int, optional
        Random seed for reproducibility, by default 0.
    verbose : int, optional
        Verbosity level of the optimizer, by default 2.

    Returns
    -------
    tuple[float, dict[str, float]]
        Best score and best hyperparameters found.
    """
    func = partial(func, train_dset=train_dset, val_dset=val_dset)
    optimizer = BayesianOptimization(
        f=func, pbounds=bounds, random_state=seed, verbose=verbose
    )
    optimizer.maximize(
        init_points=n_init_points,
        n_iter=n_iter,
    )
    return optimizer, optimizer.max["target"], (
        optimizer.max["params"]["lr"],
        int(optimizer.max["params"]["num_epochs"]),
        optimizer.max["params"]["gradient_clipping_value"],
        int(optimizer.max["params"]["batch_size"]),
        optimizer.max["params"]["mse_loss_weight"],
        optimizer.max["params"]["eikonal_loss_weight"],
        optimizer.max["params"]["laplace_loss_weight"],
        int(optimizer.max["params"]["patience"]),
        int(optimizer.max["params"]["use_elevation"]),
        int(optimizer.max["params"]["layers"]),
        float(optimizer.max["params"]["scale"]),
        int(optimizer.max["params"]["sorting_group_size"]),
        int(optimizer.max["params"]["sorting_group_size"])*int(optimizer.max["params"]["hidden_dim"]),
    )


def run_single_experiment(d: str):
    train_dset = xr.open_dataset(
        DSET_PATH / f"ecad_obs_europe_train_{d}.nc"
    ).cm
    val_dset = xr.open_dataset(DSET_PATH / f"ecad_obs_europe_val_{d}.nc").cm
    optimizer, best_loss, (
        lr,
        num_epochs,
        gradient_clipping_value,
        batch_size,
        mse_loss_weight,
        eikonal_loss_weight,
        laplace_loss_weight,
        patience,
        use_elevation,
        layers,
        scale,
        sorting_group_size,
        hidden_dim,
    ) = find_hyperparameters(
        train_dset,
        val_dset,
        compute_criterion,
        BOUNDS,
        n_init_points=OPTIM_INIT_POINTS,
        n_iter=OPTIM_N_ITERS,
        seed=SEED,
        verbose=2,
    )
    console.print("[bold yellow]Optimized parameters:[/bold yellow]")
    console.print("[yellow]Learning rate (lr):[/yellow]", lr)
    console.print("[yellow]Number of epochs:[/yellow]", num_epochs)
    console.print(
        "[yellow]Gradient clipping value:[/yellow]", gradient_clipping_value
    )
    console.print("[yellow]Batch size:[/yellow]", batch_size)
    console.print("[yellow]MSE loss weight:[/yellow]", mse_loss_weight)
    console.print("[yellow]Eikonal loss weight:[/yellow]", eikonal_loss_weight)
    console.print("[yellow]Laplace loss weight:[/yellow]", laplace_loss_weight)
    console.print(
        "[yellow]Early stopping patience:[/yellow]", patience
    )
    console.print("[yellow]Use elevation:[/yellow]", use_elevation)
    console.print("[yellow]Scale:[/yellow]", scale)
    console.print("[yellow]Hidden dims:[/yellow]", hidden_dim)
    console.print("[yellow]Layers:[/yellow]", layers)
    console.print("[yellow]Sorting_group_size:[/yellow]", sorting_group_size)
    console.print("[yellow]Best loss:[/yellow]", best_loss)
    reconstructed_dset = train_dset.reconstruct(
        val_dset.domain,
        method="sinet",
        layers=layers,
        scale=scale,
        sorting_group_size=sorting_group_size,     
        hidden_dim=hidden_dim,
        lr=lr,
        num_epochs=num_epochs,
        batch_size=batch_size,
        num_workers=0,
        device="cuda",
        gradient_clipping_value=gradient_clipping_value,
        mse_loss_weight=mse_loss_weight,
        eikonal_loss_weight=eikonal_loss_weight,
        laplace_loss_weight=laplace_loss_weight,
        patience=patience,
        use_elevation=use_elevation
    )
    cmp = cm.Comparison(reconstructed_dset, val_dset)
    metrics = cmp.compute_report()
    metrics["dataset_id"] = d
    hyperparams = {
        "dataset_id": d,
        "lr": lr,
        "num_epochs": num_epochs,
        "gradient_clipping_value": gradient_clipping_value,
        "batch_size": batch_size,
        "mse_loss_weight": mse_loss_weight,
        "eikonal_loss_weight": eikonal_loss_weight,
        "laplace_loss_weight": laplace_loss_weight,
        "patience": patience,
        "use_elevation": use_elevation,
        "layers": layers,
        "hidden_dim": hidden_dim,
        "scale": scale,
        "sorting_group_size": sorting_group_size,
        "opt_loss": best_loss,
    }
    return (optimizer, metrics, hyperparams)

In [11]:
optimizer, metrics, hyperparams = run_single_experiment(dset_idx[IDX])

|   iter    |  target   | batch_... | eikona... | gradie... | hidden... | laplac... |  layers   |    lr     | mse_lo... | num_ep... | patience  |   scale   | sortin... | use_el... |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
29-07-2025 10:39:15 INFO | climatrix.reconstruct.sinet.sinet | Initializing SiNET model...
29-07-2025 10:39:15 INFO | climatrix.reconstruct.sinet.sinet | Configuring Adam optimizer with learning rate: 0.001871
29-07-2025 10:39:15 INFO | climatrix.reconstruct.sinet.sinet | Training SiNET model...
29-07-2025 10:39:18 INFO | climatrix.reconstruct.sinet.sinet | Reconstructing target domain...
29-07-2025 10:39:18 INFO | climatrix.reconstruct.sinet.sinet | Creating mini-batches for surface reconstruction...
29-07-2025 10:39:18 INFO | climatrix.reconstruct.sinet.sinet | Processing mini-batch 1/1...
29-07-2025 10:39:18 INFO | climatrix.

Optimized parameters:

Learning rate (lr): 0.007317984048790126

Number of epochs: 490

Gradient clipping value: 3810.2685993581454

...

Hidden dims: 38

Layers: 1

Sorting_group_size: 2

Best loss: -1.4684022665023804

29-07-2025 10:49:46 INFO | climatrix.reconstruct.sinet.sinet | Initializing SiNET model...
29-07-2025 10:49:46 INFO | climatrix.reconstruct.sinet.sinet | Configuring Adam optimizer with learning rate: 0.007318
29-07-2025 10:49:46 INFO | climatrix.reconstruct.sinet.sinet | Training SiNET model...
29-07-2025 10:49:49 INFO | climatrix.reconstruct.sinet.sinet | Reconstructing target domain...
29-07-2025 10:49:49 INFO | climatrix.reconstruct.sinet.sinet | Creating mini-batches for surface reconstruction...
29-07-2025 10:49:49 INFO | climatrix.reconstruct.sinet.sinet | Processing mini-batch 1/1...
29-07-2025 10:49:49 INFO | climatrix.reconstruct.sinet.sinet | Surface finding complete. Concatenating results.


In [12]:
sinet_val_dset, sinet_reconstructed_dset, sinet_reconstructed_dense = (
    run_single_method(
        dset_idx[IDX],
        IDX,
        "sinet",
        lr=hyperparams["lr"],
        num_epochs=hyperparams["num_epochs"],
        batch_size=hyperparams["batch_size"],
        num_workers=0,
        device="cuda",
        gradient_clipping_value=hyperparams["gradient_clipping_value"],
        mse_loss_weight=hyperparams["mse_loss_weight"],
        eikonal_loss_weight=hyperparams["eikonal_loss_weight"],
        laplace_loss_weight=hyperparams["laplace_loss_weight"],
        patience=hyperparams["patience"],
        use_elevation=hyperparams["use_elevation"],
        scale=hyperparams["scale"],
        layers=hyperparams["layers"],
        sorting_group_size=hyperparams["sorting_group_size"],
        hidden_dim=hyperparams["hidden_dim"],
    )
)

29-07-2025 10:49:49 INFO | climatrix.reconstruct.sinet.sinet | Using checkpoint path: /storage/tul/projects/climatrix/experiments/jwalczak/01_Apr_02_compare_recon_method/notebooks/checkpoint
29-07-2025 10:49:49 INFO | climatrix.reconstruct.sinet.sinet | Initializing SiNET model...
29-07-2025 10:49:49 INFO | climatrix.reconstruct.sinet.sinet | Configuring Adam optimizer with learning rate: 0.007318
29-07-2025 10:49:49 INFO | climatrix.reconstruct.sinet.sinet | Training SiNET model...
29-07-2025 10:49:56 INFO | climatrix.reconstruct.sinet.sinet | Reconstructing target domain...
29-07-2025 10:49:56 INFO | climatrix.reconstruct.sinet.sinet | Creating mini-batches for surface reconstruction...
29-07-2025 10:49:56 INFO | climatrix.reconstruct.sinet.sinet | Processing mini-batch 1/1...
29-07-2025 10:49:56 INFO | climatrix.reconstruct.sinet.sinet | Surface finding complete. Concatenating results.
29-07-2025 10:49:56 INFO | climatrix.reconstruct.sinet.sinet | Using checkpoint path: /storage/tul

In [13]:
cm.Comparison(sinet_val_dset, sinet_reconstructed_dset).compute_report()


{
    'RMSE': 2.9844419956207275,
    'MAE': 1.570312261581421,
    'Max Abs Error': 14.57796859741211,
    'R^2': 0.48885273933410645
}

In [20]:
cm.Comparison(sinet_val_dset, sinet_reconstructed_dset).compute_report()


{
    'RMSE': 2.799283742904663,
    'MAE': 1.3948124647140503,
    'Max Abs Error': 14.306488037109375,
    'R^2': 0.5503096580505371
}

In [ ]:
cm.Comparison(sinet_val_dset, sinet_reconstructed_dset).compute_report()